## Instruction:
Below is an algorithm for playing the hangman game using a Hidden Markov Model. 
You can train this algorithm on your own corpus of words and test this algorithm on a separate corpus of words. 

In [19]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

In [20]:
# Class containing the algorithm 

class HangmanHMM:
    def __init__(self, weights, training_words):
        self.letter_frequency = {}
        self.letter_transition_prob_n_gram = {}
        self.letter_transition_prob_midgram = {}
        
        for n in range(-6, 7):
            if n != 0:
                self.letter_transition_prob_n_gram[n] = {}
        
        for n in range(3, 8):
            self.letter_transition_prob_midgram[n] = {}
        
        self.weights = weights
        self.training_words = training_words
    
    def train(self):
        for word in self.training_words:
            self.update_letter_frequency(word)
            self.update_transition_probabilities(word)

        self.convert_counts_to_probabilities(len(self.training_words))
        
    def update_letter_frequency(self, word):
        for letter in word:
            if letter not in self.letter_frequency:
                self.letter_frequency[letter] = 0
            self.letter_frequency[letter] += 1

    def update_transition_probabilities(self, word):
        word_length = len(word)
        
        for i in range(word_length):  
            for j in range(-6, 7):
                if j != 0:
                    self.update_transition_prob_gram(word, i, j)
    
            for k in range(3,8):
                self.update_transition_prob_midgram(word, i, k)

    def update_transition_prob_gram(self, word, index, gram_length):
        # Forward Gram Transition Probabilities
        if gram_length > 0:
            if index < len(word) - gram_length:
                gram = "".join(word[index : index + gram_length])
                next_letter = word[index + gram_length]
                transition_prob = self.get_transition_prob_dict(gram_length)
                if gram not in transition_prob:
                    transition_prob[gram] = {}
                if next_letter not in transition_prob[gram]:
                    transition_prob[gram][next_letter] = 0
                transition_prob[gram][next_letter] += 1
                
        # Backward Gram Transition Probabilities
        else:
            if index >= -gram_length:
                gram = "".join(word[index + gram_length + 1 : index + 1])
                next_letter = word[index + gram_length]
                transition_prob = self.get_transition_prob_dict(gram_length)
                if gram not in transition_prob:
                    transition_prob[gram] = {}
                if next_letter not in transition_prob[gram]:
                    transition_prob[gram][next_letter] = 0
                transition_prob[gram][next_letter] += 1
        

    def update_transition_prob_midgram(self, word, index, midgram_length):
        # Middle Gram Transition Probabilities
        if index <= len(word) - midgram_length:
            mid_gram_word = word[index : index + midgram_length]
            for mgl in range(1, midgram_length-1):
                mid_letter = mid_gram_word[mgl]
                mid_gram = list(mid_gram_word)
                mid_gram[mgl] = "_"
                mid_gram = "".join(mid_gram)
                transition_prob = self.letter_transition_prob_midgram[midgram_length]
                if mid_gram not in transition_prob:
                    transition_prob[mid_gram] = {}
                if mid_letter not in transition_prob[mid_gram]:
                    transition_prob[mid_gram][mid_letter] = 0
                transition_prob[mid_gram][mid_letter] += 1


    def get_transition_prob_dict(self, gram_length):
        return self.letter_transition_prob_n_gram[gram_length]

    def convert_counts_to_probabilities(self, total_words):
        self.convert_dict_to_probabilities(self.letter_frequency, total_words)
        
        for ngram in range(-6, 7):
            if ngram != 0:
                self.convert_transition_prob_dict_to_probabilities(self.letter_transition_prob_n_gram[ngram])
  
        for midgram in range(3, 8):
            self.convert_transition_prob_dict_to_probabilities(self.letter_transition_prob_midgram[midgram])

    def convert_dict_to_probabilities(self, data_dict, total_count):
        for key in data_dict:
            data_dict[key] /= total_count

    def convert_transition_prob_dict_to_probabilities(self, transition_prob_dict):
        for letter in transition_prob_dict:
            total_transitions = sum(transition_prob_dict[letter].values())
            for next_letter in transition_prob_dict[letter]:
                transition_prob_dict[letter][next_letter] /= total_transitions
                    
    def capture_forward_ngrams(self, word, n):
        word = "".join(word).replace(" ", "")
        ngrams = []
        for idx in range(0, len(word) - n):
            valid = 0
            ngram = "".join(word[idx : idx + n+1])

            if ngram[-1] != "_" or "_" in "".join(ngram[:n]):
                continue

            ngrams.append(ngram[:len(ngram)-1])

        return ngrams

    def capture_backward_ngrams(self, word, n):
        word = "".join(word).replace(" ", "")
        ngrams = []
        for idx in range(n, len(word)):
            ngram = "".join(word[idx-n : idx+1])

            if ngram[0] != "_" or "_" in "".join(ngram[1:]):
                continue

            ngrams.append(ngram[1:])

        return ngrams

    def capture_midgrams(self, word, n):
        word = "".join(word).replace(" ", "")
        ngrams = []

        for idx in range(len(word)-n+1):
            ngram = "".join(word[idx : idx+n])

            if ngram[0] == "_" or ngram[-1] == "_" or len(ngram.replace("_", "")) != n-1:
                continue

            ngrams.append(ngram)

        return ngrams

        
    def calculate_score(self, letter, guessed_word):
        if letter not in self.letter_frequency:
            return float(statistics.mean(list(self.letter_frequency.values())))

        score = self.weights["frequency"][0] * self.letter_frequency[letter]

        score += self.calculate_forward_score(letter, guessed_word)
        score += self.calculate_backward_score(letter, guessed_word)
        score += self.calculate_midgram_score(letter, guessed_word)

        return score

    def calculate_forward_score(self, letter, guessed_word):
        forward_score = 0.0
        word_length = len(guessed_word)
        ngrams_dict = {}
        
        for i in range(1,7):
            ngrams_dict[i] = {}
            
        for n in range(1, 7):
            if(n+1 > word_length):
                break
            ngrams = self.capture_forward_ngrams(guessed_word, n)
            ngrams_dict[n] = ngrams
            
        for n in range(1, 7):
            if(n+1 > word_length):
                break
            for gram in ngrams_dict[n]:
                g = "".join(gram)
                if g in self.letter_transition_prob_n_gram[n] and letter in self.letter_transition_prob_n_gram[n][g]:
                    forward_score += self.weights["forward"][n-1] * self.letter_transition_prob_n_gram[n][g][letter]
        
        return forward_score

    def calculate_backward_score(self, letter, guessed_word):
        backward_score = 0.0
        word_length = len(guessed_word)
        ngrams_dict = {}
        
        for i in range(1,7):
            ngrams_dict[-i] = {}
            
        for n in range(1, 7):
            ngrams = self.capture_backward_ngrams(guessed_word, n)
            ngrams_dict[-n] = ngrams
            
        for n in range(1, 7):
            for gram in ngrams_dict[-n]:
                g = "".join(gram)
                if g in self.letter_transition_prob_n_gram[-n] and letter in self.letter_transition_prob_n_gram[-n][g]:
                    backward_score += self.weights["backward"][n-1] * self.letter_transition_prob_n_gram[-n][g][letter]
        
        return backward_score

    def calculate_midgram_score(self, letter, guessed_word):
        midgram_score = 0.0
        word_length = len(guessed_word)
        ngrams_dict = {}
        
        for i in range(3, 8):
            ngrams_dict[i] = {}  
            
        for n in range(3, 8):
            ngrams = self.capture_midgrams(guessed_word, n)
            ngrams_dict[n] = ngrams
            
        for n in range(3, 8):
            for gram in ngrams_dict[n]:
                g = "".join(gram)
                if g in self.letter_transition_prob_midgram[n] and letter in self.letter_transition_prob_midgram[n][g]:
                    midgram_score += self.weights["middle"][n-3] * self.letter_transition_prob_midgram[n][g][letter]
        
        return midgram_score
    
    def guess_next_letter(self, word, guessed_letters):
        # Clean word 
        word = "".join(word).replace(" ", "")
        # Guess vowels first if no correct letters have been guessed
        if len(word.replace("_", "")) == 0:
            vowels = ['e', 'i', 'a', 'o', 'u', 'y']
            for vowel in vowels:
                if vowel not in guessed_letters:
                    return vowel

        letter_scores = {}
        for letter in list(set("abcdefghijklmnopqrstuvwxyz") - set(guessed_letters)):
            score = self.calculate_score(letter, word)
            letter_scores[letter] = score

        sorted_scores = sorted(letter_scores.items(), key=lambda x: x[1], reverse=True)
        best_guess = sorted_scores[0][0]
            
        return best_guess

In [21]:
# Building the algorithm

class Hangman(object):
    def __init__(self, full_dictionary_location):
        self.guessed_letters = []
        self.full_dictionary_location = full_dictionary_location
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        wt = {"frequency":[1], "forward":[1,3,5,7,9,11], "middle":[5,7,9,11,13], "backward":[1,3,5,7,9,11]}
        self.hangman_hmm = HangmanHMM(weights = wt, training_words = self.full_dictionary)  # Initialize HangmanHMM        
        self.hangman_hmm.train() 
        
        self.current_dictionary = []

    def build_dictionary(self, file_path):
        with open(file_path, 'r') as file:
            word_list = [word.strip() for word in file]
        
        return word_list
    
    # Function to guess next letter
    
    def guess(self, word): # word input example: "_ p p _ e "
                
        guessed_letter = self.hangman_hmm.guess_next_letter(word, self.guessed_letters)
        
        return guessed_letter

    

In [31]:

# Training the Hangman algorithm on your own corpus

train_file = "train_words.txt"
hm = Hangman(full_dictionary_location = train_file)


In [32]:
def play_hangman(word, max_wrong_guesses=5, hangman_object = None):
    partial_word = ['_'] * len(word)
    available_letters = set('abcdefghijklmnopqrstuvwxyz')
    wrong_guesses = 0
    hangman_object.guessed_letters = ""
    while wrong_guesses < max_wrong_guesses:
        # print("Partial Word:", ' '.join(partial_word))
        # print("Available Letters:", ' '.join(available_letters))

        # Make a guess based on the N-gram language model
        ngram_guess = hm.guess(word)
        # print("Guess:", ngram_guess)
        hangman_object.guessed_letters += ngram_guess
        # Remove the guessed letter from available letters
        available_letters.remove(ngram_guess)

        # Update the partial word based on the guessed letter
        for i, letter in enumerate(word):
            if letter == ngram_guess:
                partial_word[i] = letter

        # Check if the word has been completely guessed
        if '_' not in partial_word:
            # print("Congratulations! You guessed the word:", word)
            return True

        # Increment the wrong guess count if the guessed letter is incorrect
        if ngram_guess not in word:
            wrong_guesses += 1
            # print("Wrong Guesses:", wrong_guesses)

    # print("Game Over! The word was:", word)
    return False


In [34]:
# Testing the Hangman algorithm on a separate corpus

test_file = "words_250000_test.txt"
with open(test_file, 'r') as file:
    test_corpus = [word.strip() for word in file]

wins = 0
test_words = test_corpus
for i in test_words:
    word = i 
    result = play_hangman(word, max_wrong_guesses = 5, hangman_object = hm)
    wins += int(result)

win_rate = wins/len(test_words)
losses = len(test_words) - wins

print(f"Wins = {wins}")
print(f"Losses = {losses}")
print(f"Win Rate = {win_rate}")